# Solving Wordle Puzzle with Python

Run all cells below. After running the script below, it suggests you and based on the feedback you tell it, it will trim the list of words and suggest you another one.

For feedbacks you must use one of the following characters:

- c: If the character is correct in that place (green in Wordle)
- x: If the character is not in the word (grey in wordle)
-  The character is in the word, but in wrong place (yellow in wordle)

In [ ]:
digit_list = "0123456789"
char_list = "0123456789+-/*="
NERDLE_LENGTH = 8


def find(l, c):
    if c in l:
        return l.index(c)
    return 1000000


def find_indices(arr, val):
    return [i for i, x in enumerate(arr) if x == val]


def calculate(s: str) -> int:
    s = s.replace(' ', '')

    val = 0
    operations = ('+', '-', '*', '/')

    l = []

    for c in s:
        if c in operations:
            l.append(val)
            l.append(c)
            val = 0
        else:
            val = val * 10 + int(c)

    l.append(val)
    while '/' in l or '*' in l:
        div = find(l, '/')
        mul = find(l, '*')

        if div < mul:
            new_val = l[div - 1] / l[div + 1] if l[div + 1] != 0 else 1000000
            l = l[:div - 1] + [new_val] + l[div + 2:]
        else:
            new_val = l[mul - 1] * l[mul + 1]
            l = l[:mul - 1] + [new_val] + l[mul + 2:]

    ans = 0
    operator = '+'
    for op in l:
        if op in ('+', '-'):
            operator = op
        else:
            if operator == '+':
                ans += op
            else:
                ans -= op

    return ans


def validate(guess):
    gstring = "".join(guess)
    if len(gstring) != NERDLE_LENGTH or gstring.count("=") != 1:
        return False

    pre, after = gstring.split("=")
    return calculate(pre) == calculate(after)


def backtrack(guess, pos, i):
    if i == NERDLE_LENGTH:
        return validate(guess)

    for c in pos[i]:
        if i == 0 or i == NERDLE_LENGTH - 1:
            guess[i] = c
        else:
            if not guess[i - 1].isdigit() and not c.isdigit():
                continue

            if c == '0' and guess[i - 1] in '0/':
                continue

            guess[i] = c

        if backtrack(guess, pos, i + 1):
            return True

    return False


def check_result(result):
    return len(result) == NERDLE_LENGTH and all(c in "x*c-" for c in result)


pos = [set(char_list) if 0 < i < NERDLE_LENGTH - 1 else set(digit_list) for i in range(NERDLE_LENGTH)]
guess = "34+56=90"

print('Try: ' + guess)
guess = list(guess)

while True:

    result = input("Enter result: ")
    while not check_result(result):
        result = input("Enter result: ")

    if all(c == 'c' for c in result):
        print("You win!")
        break

    for i, r in enumerate(result):
        char = guess[i]
        if r == 'c':
            pos[i] = set(char)

            if char == '=':
                for j in range(NERDLE_LENGTH):
                    if j != i:
                        pos[j].discard('=')

        elif r == 'x':
            ind_list = find_indices(guess, char)

            if len(ind_list) == 1 or all(result[j] == 'x' for j in ind_list):
                for j in range(NERDLE_LENGTH):
                    pos[j].discard(char)
            else:
                pos[i].discard(char)

        elif r == '*':
            pos[i].discard(char)

    backtrack_result = backtrack(guess, pos, 0)

    if not backtrack_result:
        print('Sorry, no result was found!')
        break
    else:
        print('Try: ' + ''.join(guess))
